In [1]:
# %%writefile Generate_Base_Data.py

import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import pandas as pd  
import numpy as np  

import tools.Sample_Tools as smpl
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)
import warnings

from base.JuUnits import parallal_task,task_chunk_split
from base.JuUnits import excute_for_multidates


from QUANTAXIS.QAUtil import DATABASE
from QUANTAXIS.QAUtil import  trade_date_sse
from QUANTAXIS.QAUtil.QADate_trade import (
    QA_util_get_pre_trade_date,
    QA_util_get_next_trade_date,
    QA_util_if_tradetime
)

%load_ext autoreload
%autoreload 2
%aimport tools.Cacher,tools.Sample_Tools

def cache_single_stock_qfq(codes,self_obj=None):
    import tools.Sample_Tools as smpl
    from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)
    import warnings
    
    for code in codes:
        try:
            stock_df = smpl.get_data([code], start='2000-01-01', end='2023-01-01').to_qfq().data
            stock_df = smpl.add_marketvalue_industry(stock_df)
            dt_index = stock_df.index.get_level_values(0)
        except:
            # warnings.warn('get "{}" error'.format(code))
            continue

        try:
            save_cache('{}_train_qfq'.format(code) ,stock_df.loc[(dt_index >= '2017') & (dt_index < '2022')], cache_type=CACHE_TYPE.STOCK)
            save_cache('{}_tail_qfq'.format(code), stock_df.loc[(dt_index >= '2022')], cache_type=CACHE_TYPE.STOCK)
        except:
            warnings.warn('1.save_cache "{}" error'.format(code))
            continue

        try:
            save_cache('{}_older_qfq'.format(code), stock_df.loc[dt_index < '2017'],cache_type=CACHE_TYPE.STOCK)
        except:
            warnings.warn('2.save_cache "{}" error'.format(code))
            continue
        # print('{} saved'.format(code))
    

def generate_single_stock_qfq(codes, worker=6):
    task = task_chunk_split(codes, worker)
    results = parallal_task(worker, cache_stock_qfq, task)
    
    

def cache_all_stock_qfq(codes,self_obj=None):
    import tools.Sample_Tools as smpl
    import warnings
    
    try:
        stock_df = smpl.get_data(codes, start='2010-01-01', end='2023-01-01').to_qfq().data
        stock_df = smpl.add_marketvalue_industry(stock_df)
        dt_index = stock_df.index.get_level_values(0)
    except:
        warnings.warn('get "{}" error'.format(codes[0]))
        
    return stock_df

    

def generate_all_stock_qfq(codes, worker=6):
    task = task_chunk_split(codes, worker)
    results = parallal_task(worker, cache_stock_qfq, task)
    results = pd.concat(results).sort_index()
    dt_index = results.index.get_level_values(0)
    try:
        save_cache('all_train_qfq' ,results.loc[(dt_index >= '2017') & (dt_index < '2022')], cache_type=CACHE_TYPE.STOCK)
        save_cache('all_tail_qfq', results.loc[(dt_index >= '2022')], cache_type=CACHE_TYPE.STOCK)
    except:
        warnings.warn('1.save_cache error')

    try:
        save_cache('all_older_qfq', results.loc[dt_index < '2017'],cache_type=CACHE_TYPE.STOCK)
    except:
        warnings.warn('2.save_cache error')

In [15]:
# codes = smpl.get_codes_from_blockname('沪深300', sse='all')
codes = smpl.get_codes_by_market(sse='all',only_main=True,filter_st=True)
generate_all_stock_qfq(codes)


Now in the main code. Process name is: base.JuUnits
base.JuUnits, subpid:5812  pid:9392


  0%|          | 0/6 [00:00<?, ?it/s]

In [16]:
df_all = pd.concat(list(map(lambda file:load_cache(file,cache_type=CACHE_TYPE.STOCK),['all_train_qfq','all_tail_qfq','all_older_qfq']))).sort_index()
df_all

open       high        low      close    volume  \
date       code                                                           
2010-01-04 000001   7.733425   7.752348   7.468495   7.477957  241922.0   
           000002   6.955789   6.968610   6.795517   6.795517  969832.0   
           000006   3.191170   3.196803   3.129205   3.132022   62998.0   
           000007   4.726667   4.726667   4.600000   4.620000   25901.0   
           000009   4.764659   4.777653   4.678028   4.695354  173626.0   
...                      ...        ...        ...        ...       ...   
2022-12-30 605580  11.890000  12.080000  11.800000  12.070000    7771.0   
           605588  28.590000  28.800000  28.440000  28.800000    2557.0   
           605589  21.700000  21.870000  20.980000  21.390000   77000.0   
           605598  25.300000  25.500000  24.670000  24.900000   22984.0   
           605599   9.970000  10.230000   9.900000  10.180000   65106.0   

                         amount       adj        shares        lshares  \
date       code                                                          
2010-01-04 000001  5.802495e+08  0.315393  3.105434e+05  292411.437500   
           000002  1.034345e+09  0.641087  1.099521e+06  965609.500000   
           000006  7.054856e+07  0.281657  5.071833e+04   49332.269531   
           000007  1.810142e+07  0.666667  1.849654e+04   18496.537109   
           000009  1.894014e+08  0.433151  1.038810e+05   94106.656250   
...                         ...       ...           ...            ...   
2022-12-30 605580  9.326484e+06  1.000000  2.000000e+04    5111.000000   
           605588  7.322457e+06  1.000000  7.309956e+03    2309.956055   
           605589  1.638851e+08  1.000000  7.828768e+04   61521.179688   
           605598  5.754628e+07  1.000000  1.727435e+04    5264.346680   
           605599  6.582705e+07  1.000000  7.777778e+04   58667.781250   

                   totalCapital  liquidity_totalCapital industry  
date       code                                                   
2010-01-04 000001  2.322230e+10            2.186640e+10    股份制银行  
           000002  7.471814e+10            6.561816e+10     住宅开发  
           000006  1.588509e+09            1.545097e+09     住宅开发  
           000007  8.545400e+08            8.545400e+08   商业物业经营  
           000009  4.877581e+09            4.418641e+09    电池化学品  
...                         ...                     ...      ...  
2022-12-30 605580  2.414000e+09            6.168977e+08     热力服务  
           605588  2.105267e+09            6.652673e+08     光学元件  
           605589  1.674573e+10            1.315938e+10     合成树脂  
           605598  4.301313e+09            1.310822e+09   其他专业工程  
           605599  7.917778e+09            5.972380e+09     钟表珠宝  

[7081385 rows x 12 columns]